# INTRODUCTION TO THE BACKWARD PASS

A Backward Pass in Neural Networks can be used for optimization of losses using gradient descent. 
I will refer to the Pytorch Module, in it loss.backward() seems a similar methodology that we are studying.

The Purpose of this project  is to demonstrate how to perform a backward pass on Quantum Neural Networks.As you might recall
from previous notebooks ,there are two type of Networks : And their implementation will also differ slightly.It is worth to 
Mention that there is a slight difference between the Estiamtor QNN and Sampler QNN ,that is while Estimator QNN takes
in observables ,the SamplerQNN feed directly into a quantum circuit.Also The Output in a Sampler QNN depends on whether or not we 
have an interpret function
The Backward Pass is then calculated with repect to the input and weights .It returns a tuple of input_gradients and weight gradients



In [ ]:
# importing dependencies
import qiskit
from qiskit import QuantumCircuit,QuantumRegister
import qiskit_machine_learning
from qiskit_machine_learning.neural_networks import SamplerQNN,EstimatorQNN
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import Parameter,ParameterVector

from qiskit.utils import algorithm_globals
algorithm_globals.random_seed= 42

# ESTIMATOR QNN :: BACKWARD PASS

In [ ]:
# Lets define the parameters for our estimator qnn
# Create a quantum circuit
qc1 = QuantumCircuit(2)
parameters = [Parameter("estim_input"),Parameter("estim_weight")]
qc1.h(0)
qc1.ry(parameters[0],0)
qc1.rx(parameters[1],1)
qc1.draw("mpl")

In [ ]:
# Create an Observable ,
observable = SparsePauliOp.from_list([("Y"*qc1.num_qubits,1)])

In [ ]:
# Instantiate our Estimator QNN
estimator_qnn = EstimatorQNN(observables=observable,circuit=qc1,input_params=[parameters[0]],weight_params = [parameters[1]])

In [ ]:
estimator_qnn.weight_params

In [ ]:
# Now We can create some dummy data using algorithm globals 
estimator_qnn_input = algorithm_globals.random.random(estimator_qnn.num_inputs)
estimator_qnn_weights = algorithm_globals.random.random(estimator_qnn.num_weights)

In [ ]:
estimator_qnn_input

# Backward Pass without Input_Gradients 
The Output Shape for Estimator QNN = (BATCH_SIZE,NUM_QUBITS*NUM_OBSERVABLES,NUM_WEIGHTS)

In [ ]:
e_qnn_input_grad,e_qnn_weight_grad = estimator_qnn.backward(estimator_qnn_input,estimator_qnn_weights)
print(f"The Estimator QNN input gradients are {e_qnn_input_grad} and the shape is \n Shape:{e_qnn_input_grad}")
print(f"The Estimator QNN input gradients are {e_qnn_weight_grad} and the shape is \n{e_qnn_weight_grad.shape}")

# Backward Pass with Input_Gradients 

In [ ]:
# Set the Input Gradients to be equal to True
estimator_qnn.input_gradients = True # Has a similarity to model.forward.lstm or model.forward.>>

In [ ]:
e_qnn_input_grad,e_qnn_weight_grad = estimator_qnn.backward(estimator_qnn_input,estimator_qnn_weights)
print(f"The Estimator QNN input gradients are {e_qnn_input_grad} and the shape is \nShape:{e_qnn_input_grad.shape}")
print(f"The Estimator QNN Weight gradients are {e_qnn_weight_grad} and the shape is \n{e_qnn_weight_grad.shape}")

# SAMPLER QNN BACKWARD PASS 

In [ ]:
# Lets define our quantum_circuit
qc2 = QuantumCircuit(2)
sampler_input = ParameterVector("input",2)
sampler_weight  = ParameterVector("weight",4)
qc2.h(0)
qc2.ry=(sampler_input[0],0)
qc2.ry=(sampler_input[1],1)
qc2.cx(0,1)
qc2.ry= (sampler_weight[0],0)
qc2.ry= (sampler_weight[1],1)
qc2.cx(0,1)
qc2.ry = (sampler_weight[2],0)
qc2.ry = (sampler_weight[3],1)
qc2.draw("mpl")

In [ ]:
sampler_qnn = SamplerQNN(circuit=qc2,input_params=sampler_input,weight_params=sampler_weight)

In [ ]:
sampler_input = algorithm_globals.random.random(sampler_qnn.num_inputs)
sampler_weights = algorithm_globals.random.random(sampler_qnn.num_weights)

In [ ]:
sampler_input

In [ ]:
sampler_weights

In [ ]:
sampler_input.shape ,sampler_weights.shape

In [ ]:
s_qnn_input_grad,s_qnn_weight_grad = sampler_qnn.forward(sampler_input,sampler_weights)

In [ ]:
s_qnn_input_grad,s_qnn_weight_grad = sampler_qnn.backward(sampler_input,sampler_weights)

In [ ]:
class NeuralNetworks(qiskit_machine_learning.neural_networks):
    pass

# REFERENCES
https://qiskit.org/ecosystem/machine-learning/tutorials/01_neural_networks.html